# Analyse en composantes principales

## Préliminaires

Nous allons avoir besoin pour le TP des bibliothèques suivantes (pas seulement pour l'algèbre):

In [1]:
import numpy as np
import numpy.random as ra
from scipy import linalg as lin
import matplotlib.pyplot as plt

### L'algèbre linéaire avec Numpy

Dans ce TP nous sommes amenés à effectuer de nombreux produits matriciels et scalaires. Voici quelques règles concernant Numpy:

* Le produit scalaire dans Numpy peut se faire via `np.dot(x,y)`, ou dans Python 3, via `x @ y`.
* Dans Numpy tous les vecteurs sont des vecteurs lignes, mais `y` dans les formules ci-dessus est interprété comme un vecteur colonne (avec comme conséquence que `@` n'est pas toujours associatif).
* De même, un opérateur matriciel `A` peut être appliqué à `y` via `np.dot(A,y)`, ou dans Python 3, via `A @ y`. Ici encore, `y` est interprété comme un vecteur colonne.
* Le produit matriciel `np.dot(A,B)`, ou dans Python 3, `A @ B`, suit les règles habituelles.
* Un tableau peut être transposé via l'attribut `.T`, par exemple `A.T`. 
* Il est possible de multiplier un vecteur ou une matrice par un scalaire en utilisant le signe de multiplication `*`  : `B = 3.0 * A`
* Le signe `*` sur deux matrices de même taille effectue une multiplication terme à terme.


# Analyse en composantes principales

Dans cette première étude nous allons réaliser une [analyse en composantes principales](https://www.astromatic.net/m2aais/notes/Decompositions.html#analyse-en-composantes-principales) (ACP) d'un jeu de spectres de sources (étoiles, galaxies, quasars). Ce travail nous permettra de nous familiariser avec
- le calcul des [vecteurs et de valeurs propres](https://fr.wikipedia.org/wiki/Valeur_propre,_vecteur_propre_et_espace_propre) d'un opérateur,
- le concept de [décorrélation](https://fr.wikipedia.org/wiki/D%C3%A9corr%C3%A9lation) de composantes,
- la [réduction dimensionnelle](https://fr.wikipedia.org/wiki/R%C3%A9duction_dimensionnelle).

## Les données

Le fichier `spec4000_recons_2023_interp.npz` est issu de la collection d'exemples [AstroML](https://www.astroml.org) et contient les [spectres](https://fr.wikipedia.org/wiki/Spectroscopie) de 4000 sources obtenus par le relevé [SDSS](https://www.sdss.org/).
Les spectres, en unités arbitraires de [densité de flux](https://fr.wikipedia.org/wiki/Densit%C3%A9_de_flux), couvrent le domaine de longueur d'onde allant de 3000 à 8000 Å et sont corrigés d'un [décalage vers le rouge](https://fr.wikipedia.org/wiki/D%C3%A9calage_vers_le_rouge) éventuel (ramenés dans le [référentiel](https://fr.wikipedia.org/wiki/R%C3%A9f%C3%A9rentiel_(physique)) de la source).
Pour chaque source certains intervalles spectraux ont dû être retouchés car inaccessibles en raison du décalage spectral ou contaminés par des raies atmosphériques.

Le chargement des données s'opère de la façon suivante:

In [ ]:
spec = np.load("spec4000_recons_2023_interp.npz") 
print(spec.files)

On constate que le fichier contient plusieurs [tableaux NumPy](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html). Pour ce TP nous n'allons charger que les spectres (tableau de vecteurs lignes), l'échelle des longueurs d'ondes (pour les tracés), le classement de chaque source, et les étiquettes associées.

In [3]:
spectra = spec["spectra"]
wave = spec["wave"]
classes = spec["classes"]
labels = spec["labels"]
masks = spec["masks"]

### 🏋 Exercice

Tout d'abord, nous allons manipuler un peu les données.
1. Vérifier le nombre de spectres disponibles et l'échantillonnage en fréquence de chacun d'entre eux.
2. Combien de classes de spectres différents trouve-t-on dans cet ensemble (indication : on peut utiliser l'opérateur `set` pour transformer une liste en un ensemble d'éléments uniques)? Quels sont les noms de ces classes ?


In [ ]:
#1

#2


3. On peut afficher le nombre de spectres de chaque classe par ordre croissant grâce aux instructions suivantes. Assurez-vous de bien comprendre, complètement, ces instructions, et commentez le mieux possible chaque ligne.

In [ ]:
classeU = np.array(list(set(classes))) # COMMENTAIRE 
nclasse = np.array([np.sum(classes==cl) for cl in classeU]) # COMMENTAIRE 
classeU = classeU[np.argsort(nclasse)] # COMMENTAIRE 
nclasse = np.sort(nclasse) # COMMENTAIRE 
for i in range(len(classeU)):
    print(labels[classeU[i]],":",nclasse[i],"objets")


4. Pour chaque classe afficher 3 spectres (on utilisera pour les abscisses l'échelle commune des longueurs d'onde du tableau `wave`). Discutez des propriétés qualitatives. Peut-on les distinguer à l'oeil ?
5. Filtrer la base de spectres (i.e. les variables `spectra`, `classes`, `classeU` et `nclasse`) pour en retirer les spectres sans label ("unknown").

In [ ]:
#4

#5


On se débarrasse pour la suite de la classe `unknown`:

In [ ]:

if labels[classeU[0]] == 'unknown': # vérifie qu'on n'a pas déjà fait cette sélection
    print('On fait une sélection pour exclure les classifications inconnues')
    spectra = spectra[classes!=0]
    classes = classes[classes!=0]
    classeU = classeU[1:]
    nclasse = nclasse[1:]
else:
    print('Sélection déjà faite...')

## Axes principaux

### 🏋 Exercice
1. Normaliser l'amplitude de chaque spectre de sorte que la moyenne de chaque spectre vaille 1. Quel est l'intérêt d'une telle normalisation pour l'analyse en composante principales?
2. Calculer la matrice de covariance empirique corrigée associée au jeu de spectres normalisés, dans l'espace des signaux (matrice 1000×1000 attendue, c'est bien la covariance pour chaque échantillon de longueur d'onde que l'on cherche).
3. Calculer le spectre normalisé moyen et l'afficher. Le comparer à des spectres (normalisés) des différentes familles.

In [ ]:
#1

#2


#3


  


4. Extraire le spectre des valeurs propres et la matrice des vecteurs propres de la matrice de covariance (on pourra utiliser la fonction [`np.linalg.eigh()`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.eigh.html)).
5. Réorganiser le spectre des valeurs propres et la matrice des vecteurs propres par valeurs propres décroissantes (on pourra utiliser la fonction [`np.argsort()`](https://numpy.org/doc/stable/reference/generated/numpy.argsort.html)).

In [9]:
#4

#5



## Exploration de la décomposition

### 🏋 Exercice
1. Afficher le spectre de la décomposition, c'est à dire les valeurs propres par valeurs décroissantes. Commenter.

In [ ]:
# 1

2. Afficher quelques vecteurs propres (axes principaux) parmi les premiers et les derniers. Commenter. Quelle est leur moyenne ?

In [ ]:

#2



3. Vérifier que comme attendu les vecteurs propres sont orthogonaux deux à deux

In [ ]:

#3


4. Tracer fraction de variance totale expliquée $r(k)$ (somme des valeurs propres $\lambda_i$ avec $i\le k$) en fonction du nombre de composantes principales considéréees $k$ (avec $1 \le k \le 1000$) ? Que laisse augurer le comportement de $r$ en matière de réduction dimensionnelle? Quelle est la contribution des 5 premiers vecteurs propres, des 100 derniers ? Combien doit-on en conserver pour représenter 90%, 95% de l'information ? 

In [ ]:
# 4


## Projections sur les axes

### 🏋 Exercice

1. Calculer par projection sur les vecteurs propres les 2 premières composantes principales pour chacun des 4000 spectres. Montrer qu'elles sont décorrélées (on n'oubliera pas de soustraire le spectre moyen avant de réaliser les produits scalaires).


In [ ]:
#1

2. Calculer en une opération python la projection pour toutes les composantes principales. Vérifier la décorrélation.


In [ ]:

#2



## Réduction dimensionnelle et reconstruction partielle

On a vu que seuls quelques composantes dominaient le spectre et la cumulative de la matrice de covariance. Ce sont les composantes les plus informatives. On a tracé les vecteurs correspondants qui visuellement contiennent de l'information sur les spectres contrairement aux moins informatifs qui se comportaient comme du bruit (d'amplitude variable cependant). 
On peut utiliser ces propriétés pour produire des reconstructions partielles des spectres filtrées des composantes les moins informatives.

### 🏋 Exercice
1. Reconstruire partiellement tous les spectres normalisés à partir du spectre moyen et du premier vecteur propre. Reproduire les figures de la première partie et afficher pour un exemple de chaque classe le spectre, la moyenne des spectres, la reconstruction partielle et la différence avec le spectre. Commenter.


In [ ]:

#1

2. Reprendre la question avec le nombre de composante calculées au dessus pour 85% de l'information et pour 95%. On essaiera de tirer parti au maximum du calcul matriciel pour s'affranchir des boucles.Commenter.


(on essaiera de tirer parti au maximum du calcul matriciel pour s'affranchir des boucles).
3. Tracer les reconstructions des trois exemples choisis en 1 et les comparer au spectres initiaux normalisés (on pourra aussi tracer la différence pour illustrer la qualité de reconstruction).

In [ ]:
# 2



## Séparation
On a vu que selon les types d'objets, il fallait plus ou moins de composante pour les représenter efficacement.
Nous allons maintenant explorer cela sommairement en montrant le comportement joint des composantes. Dans les cas les plus simples ce genre d'exploration suffit à séparer des populations d'objets différents. Ce n'est que partiellemenent le cas ici. 

### 🏋 Exercice
1. Afficher dans un même diagramme chaque spectre comme un point d'abcisse égale à la première composante et d'ordonnée égale à la seconde composante, en choissant des couleurs par classe d'objet. On prendra soin d'afficher d'abord la classe contenant le plus d'objets afin de rendre la figure lisible ! Commenter.

2. Construire avec `plt.subplots` un triangle de ces comportements joints pour les 6 premières composantes prises 2 à 2. Commenter.




In [ ]:
#1

#2